In [45]:
import pandas as pd
dataFolder = '../Data/wikispeedia_paths-and-graph/'
articlesDf = pd.read_csv(dataFolder+ 'articles.tsv')
'''articlesDf = articlesDf.index.to_frame
articlesDf'''
articlesDf = articlesDf.iloc[16:]
articlesDf = articlesDf.index
articlesDf = articlesDf.to_frame()
articlesDf = articesDf

,,0,1
10th_century,NaN,10th_century,NaN
11th_century,NaN,11th_century,NaN
12th_century,NaN,12th_century,NaN
13th_century,NaN,13th_century,NaN
14th_century,NaN,14th_century,NaN
...,...,...,...
Zionism,NaN,Zionism,NaN
Zirconium,NaN,Zirconium,NaN
Zoroaster,NaN,Zoroaster,NaN
Zuid-Gelders,NaN,Zuid-Gelders,NaN
